In [2]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
spark = SparkSession.builder.appName('solution_ex_09_v2').getOrCreate()

22/07/18 14:38:18 WARN Utils: Your hostname, certusr-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/07/18 14:38:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/18 14:38:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
inputPath = './data'
outputPath = './output_ex_09_v2'

print(os.path.exists(inputPath))
print(os.path.exists(outputPath))

True
False


In [5]:
dfRDD = spark.read.load(inputPath, header=False, format="csv", inferschema=True)\
                            .withColumnRenamed("_c0", "sensorID")\
                            .withColumnRenamed("_c1", "date")\
                            .withColumnRenamed("_c2", "PM10")

In [6]:
dfRDD.createOrReplaceTempView('readings')

# Define one group for each value of sensorId and count the number of records with PM10>50.
# Select only the sensors with at least two high values

sensorsMaxValues = spark.sql("""
                    select sensorID, count(*)
                    from readings
                    where PM10 > 50
                    group by sensorID
                    having count(*)>1""")
sensorsMaxValues.show()

+--------+--------+
|sensorID|count(1)|
+--------+--------+
|      s1|       2|
+--------+--------+



In [7]:
sensorsMaxValues.write.csv(outputPath)
